# Raw Frequency Data → Master Curve

Apply manual shift factors to build a mastercurve and fit RheoJAX models.

In [1]:
# Google Colab compatibility - uncomment if running in Colab
# !pip install -q rheojax
# from google.colab import drive
# drive.mount('/content/drive')


## Setup and Imports
Use raw multi-temperature frequency sweeps and apply RheoJAX mastercurve shifting with PyVisco shift factors.

In [2]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
%matplotlib inline

import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from rheojax.core.data import RheoData
from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fractional_maxwell_model import FractionalMaxwellModel
from rheojax.models.generalized_maxwell import GeneralizedMaxwell
from rheojax.pipeline.base import Pipeline
from rheojax.transforms.mastercurve import Mastercurve

jax, jnp = safe_import_jax()
verify_float64()
np.set_printoptions(precision=4, suppress=True)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore', category=RuntimeWarning)

def r2_complex(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    ss_res = np.sum(np.abs(y_true - y_pred) ** 2)
    ss_tot = np.sum(np.abs(y_true - np.mean(y_true)) ** 2)
    return float(1 - ss_res / ss_tot)


INFO:2025-12-06 04:15:08,947:jax._src.xla_bridge:808: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Loading rheojax version 0.4.0


/Users/b80985/Projects/rheojax/.venv/lib/python3.13/site-packages/piblin_jax/__init__.py:28: UserWarning: GPU acceleration is only available on Linux with CUDA 12+. Using JAX in CPU mode.
  from . import backend, data, dataio, transform


## Load raw sweeps and shift factors

In [3]:
DATA_DIR = Path.cwd().parent / 'data' / 'pyvisco' / 'freq_raw'
raw_df = pd.read_excel(DATA_DIR / 'freq_Eplexor_raw.xls')
shift_df = pd.read_csv(DATA_DIR / 'freq_Eplexor_raw__shift_factors.csv')
shift_df.columns = shift_df.columns.str.strip()

# Drop header row (units) and coerce numeric
raw_clean = raw_df[pd.to_numeric(raw_df['Nr'], errors='coerce').notna()].copy()
raw_clean['T'] = raw_clean['T'].astype(float)

# Build RheoData per temperature
datasets = []
for temp_c, frame in raw_clean.groupby('T'):
    omega = 2 * np.pi * frame['f'].astype(float).to_numpy()
    Gp = frame["E'"] .astype(float).to_numpy()
    Gpp = frame["E''"].astype(float).to_numpy()
    data = RheoData(
        x=omega,
        y=Gp + 1j * Gpp,
        x_units='rad/s',
        y_units='MPa',
        domain='oscillation',
        metadata={'temperature': temp_c + 273.15, 'test_mode': 'oscillation'}
    )
    datasets.append(data)

# Manual shift factors from PyVisco table
shift_clean = shift_df.iloc[1:].copy()
shift_clean['T'] = shift_clean['T'].astype(float) + 273.15
shift_clean['log_aT'] = shift_clean['log_aT'].replace('-', 0).astype(float)
manual_shifts = {float(T): float(10 ** log_aT) for T, log_aT in zip(shift_clean['T'], shift_clean['log_aT'])}
ref_temp = float(shift_clean['T'].iloc[0])

mc = Mastercurve(reference_temp=ref_temp, method='manual', auto_shift=False)
mc.set_manual_shifts(manual_shifts)
master, shift_factors = mc.create_mastercurve(datasets, return_shifts=True)

print(f"Mastercurve built with {len(datasets)} temperatures → {len(master.x)} points")
shift_factors


Mastercurve built with 155 temperatures → 210 points


{223.1137: 1.0,
 223.14329999999998: 1.0,
 223.1735: 1.0,
 223.18829999999997: 1.0,
 223.1888: 1.0,
 223.3524: 1.0,
 223.36649999999997: 1.0,
 223.36699999999996: 1.0,
 230.55749999999998: 1.0,
 230.58739999999997: 1.0,
 230.60219999999998: 1.0,
 230.6024: 1.0,
 230.60269999999997: 1.0,
 230.6171: 1.0,
 230.61759999999998: 1.0,
 230.63179999999997: 1.0,
 230.6323: 1.0,
 238.1504: 1.0,
 238.15059999999997: 1.0,
 238.16539999999998: 1.0,
 238.16549999999998: 1.0,
 238.1658: 1.0,
 238.1803: 1.0,
 238.18039999999996: 1.0,
 238.195: 1.0,
 245.6306: 1.0,
 245.65999999999997: 1.0,
 245.66049999999998: 1.0,
 245.67499999999998: 1.0,
 245.6752: 1.0,
 245.67569999999998: 1.0,
 245.69019999999998: 1.0,
 245.70479999999998: 1.0,
 253.18769999999998: 1.0,
 253.20279999999997: 1.0,
 253.21729999999997: 1.0,
 253.21779999999998: 1.0,
 253.2181: 1.0,
 253.2326: 1.0,
 253.23289999999997: 1.0,
 253.23299999999998: 1.0,
 253.24779999999998: 1.0,
 260.67449999999997: 1.0,
 260.6747: 1.0,
 260.6898: 1.0,
 

## Plot raw vs shifted curves

In [4]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)
colors = plt.cm.viridis(np.linspace(0, 1, len(datasets)))

for color, data in zip(colors, datasets):
    temp_c = data.metadata['temperature'] - 273.15
    axes[0].loglog(data.x / (2*np.pi), np.real(data.y), 'o', color=color, label=f"{temp_c:.0f}°C E'")
    axes[0].loglog(data.x / (2*np.pi), np.imag(data.y), 's', color=color, alpha=0.6, label=f"{temp_c:.0f}°C E''")
axes[0].set_xlabel('Frequency (Hz)')
axes[0].set_ylabel('Modulus (MPa)')
axes[0].set_title('Unshifted raw sweeps')
axes[0].grid(True, which='both', ls='--', alpha=0.4)
axes[0].legend(ncol=2, fontsize=8)

axes[1].loglog(master.x / (2*np.pi), np.real(master.y), 'o', label="E' master", alpha=0.7)
axes[1].loglog(master.x / (2*np.pi), np.imag(master.y), 's', label='E" master', alpha=0.7)
axes[1].set_xlabel('Frequency (Hz, shifted)')
axes[1].set_title('Shifted mastercurve')
axes[1].grid(True, which='both', ls='--', alpha=0.4)
axes[1].legend()
plt.show()


/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_14276/1204038977.py:20: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Fit models on mastercurve

In [5]:
omega_master = master.x
G_master = master.y

# Generalized Maxwell on mastercurve
gm = GeneralizedMaxwell(n_modes=6, modulus_type='tensile')
gm.fit(omega_master, G_master, test_mode='oscillation', use_log_residuals=True)
gm_pred_components = gm.predict(omega_master)
gm_pred = gm_pred_components[:, 0] + 1j * gm_pred_components[:, 1]
gm_r2 = r2_complex(G_master, gm_pred)

fm_pred = np.full_like(G_master, np.nan)
fm_r2 = np.nan
try:
    fm = FractionalMaxwellModel()
    fm.fit(omega_master, G_master, test_mode='oscillation', use_log_residuals=True)
    fm_pred = fm.predict(omega_master, test_mode='oscillation')
    fm_r2 = r2_complex(G_master, fm_pred)
except Exception as exc:
    print(f"Fractional Maxwell fit failed: {exc}")

print(f"Generalized Maxwell R^2: {gm_r2:.4f}")
print(f"Fractional Maxwell R^2:   {fm_r2 if np.isfinite(fm_r2) else float('nan'):.4f}")


Starting least squares optimization | {'method': 'trf', 'n_params': 13, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 1.366223s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=20 | final_cost=3.323818e+08 | time=1.366s | final_gradient_norm=15666336330359.125


Starting least squares optimization | {'method': 'trf', 'n_params': 13, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.323514s


...

Element minimization: reducing from 6 to 1 modes


Starting least squares optimization | {'method': 'trf', 'n_params': 4, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.778567s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=22 | final_cost=1.560306e+02 | time=0.779s | final_gradient_norm=76517.33832053197


Generalized Maxwell R^2: 0.0048
Fractional Maxwell R^2:   -1.8143


## Overlay fits on shifted data

In [6]:
fig, ax = plt.subplots(figsize=(9, 6))
ax.loglog(master.x / (2*np.pi), np.real(master.y), 'o', label="E' data", alpha=0.5)
ax.loglog(master.x / (2*np.pi), np.imag(master.y), 's', label='E" data', alpha=0.5)
ax.loglog(master.x / (2*np.pi), np.real(gm_pred), '-', label="E' GM")
ax.loglog(master.x / (2*np.pi), np.imag(gm_pred), '--', label='E" GM')

if np.isfinite(fm_r2):
    ax.loglog(master.x / (2*np.pi), np.real(fm_pred), '-', label="E' FM")
    ax.loglog(master.x / (2*np.pi), np.imag(fm_pred), '--', label='E" FM')
else:
    ax.text(0.5, 0.1, 'Fractional fit failed', transform=ax.transAxes)

ax.set_xlabel('Frequency (Hz, shifted)')
ax.set_ylabel('Modulus (MPa)')
ax.set_title('Mastercurve fits')
ax.grid(True, which='both', ls='--', alpha=0.4)
ax.legend()
plt.show()


/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_14276/1592546360.py:18: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
